# 다나와 제조사검색 후 조회


- 노트북 카테고리에서 제조사 선택(더보기에 나오는 것) 
    - disabled 되어있어서 더보기 버튼 클릭후 클릭 가능
- 각 제품의 이름, 사진링크, 가격을 가져온다.
- 5페이지만 가져온다.

#### 주의
- 더보기 버튼
    - 다시 요청해서 가져오는 것은 아니지만 더보기 눌러야 나오는 제조사들은 disabled 되어있어서 클릭(선택이 안된다)
    - 그래서 더보기 클릭후 항목을 선택해야한다.

#### Image Lazy loading 설명
- 사진링크
    - `img['data-original]` 로 조회-> 없으면(Exception) `img[src]`

### selector
- 더보기: dl#dlMaker_simple button.btn_spec_view.btn_view_more
- Apple: dl#dlMaker_simple > dd > ul:nth-of-type(2) > li:nth-child(12) 

- 제품목록: li.prod_item.prod_layer
    - 타이틀: p.prod_name a : text
    - 가격: p.price_sect a strong : text
    - 이미지링크: div.thumb_image  img : data_orignal,  src
    
- 페이지 : div.number_wrap   a:nth-child(페이지번호)

In [2]:

import time
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

# options = ChromeOptions()
# options.add_argument("headless")

driver = Chrome()
# driver.implicitly_wait(5)
# driver.set_window_size(1920, 1280)  # maximize_window(), minimize_window()

# 다나와 노트북 페이지 이동
driver.get(url) #노트북




# 제조사별 더 보기 클릭 - 처음 로딩후 선택해야 하므로 WebDriverWait 이용
WebDriverWait(driver, 3) \
   .until(
   EC.presence_of_element_located((By.CSS_SELECTOR, 'dl#dlMaker_simple button.btn_spec_view.btn_view_more'))).click()


# 원하는 모델 카테고리 클릭 - disable되어 있지만 이미 있으므로 find로 조회(sleep후)
time.sleep(1)
apple_cb = driver.find_element_by_css_selector('dl#dlMaker_simple dd.item_dd > ul:nth-of-type(2) > li:nth-child(12) >label>input')
apple_cb.click()
# 3초간 대기 - 검색 처리 요청을 대기한다.
time.sleep(3)

# 크롤링 페이지 수
target_crawl_num = 6
# 5페이지
result_list = []
for curr_page_num in range(1, target_crawl_num):
    print(curr_page_num)
    soup = BeautifulSoup(driver.page_source, "lxml")

    # 메인 상품 리스트 선택
    # div.main_prodlist.main_prodlist_list 로 광고는 맨위 광고 productlist는 빠진다.
    # 애드포인트는 li사이에 <div>로 있으므로 안나온다.
    # 애드캐스터는 li.prod_ad_item이므로 뺀다.
    prduct_list_tags = soup.select('li.prod_item.prod_layer') # 광고 빼고 다 조회된다.

    # 제품 정보 조회
    for li_tag in prduct_list_tags:

#       제품명
        p_name = li_tag.select_one('p.prod_name > a').text.strip()
#       이미지 링크
        try:
            img_link = li_tag.select_one('a.thumb_link > img')['data-original']
        except:
            img_link = li_tag.select_one('a.thumb_link > img')['src']
#       가격
        price = li_tag.select('p.price_sect > a')[0].text.strip() #가격은 하나만 조회
        result_list.append((p_name, price, img_link))
        
#  페이지 이동 클릭 (페이지 링크는 이미 있으므로 WebDriverWait안한다.)
    next_page_btn = driver.find_element_by_css_selector('div.number_wrap > a:nth-child({})'.format(curr_page_num+1)) #다음 페이지 이므로 +1 한다.
    next_page_btn.click()
   
    time.sleep(2)

# 브라우저 종료
# driver.quit()


SessionNotCreatedException: Message: Unable to find a matching set of capabilities


In [ ]:
len(result_list)

In [ ]:
# li안에 있는 것 selector 테스트 할 때 개발자 도구에서 하면 여러개 나오므로 BS에 넣어서 해볼 수 있다.
html = """
<li class="prod_item prod_layer " id="productItem11609756">
					<input type="hidden" id="productItem_categoryInfo_11609756" value="디지털 완제품_노트북">
					<div class="prod_main_info">
						<div class="thumb_image">
															<a href="http://prod.danawa.com/info/?pcode=11609756&amp;cate=112758" class="thumb_link" target="_blank" onmousedown="_trkEventLog('15상품리스트_상품이미지_상품블로그 이동');">
																			<img src="http://img.danawa.com/prod_img/500000/756/609/img/11609756_1.jpg?shrink=160:160&amp;_v=20200716165437" alt="한성컴퓨터 올데이롱 TFX5470H이미지입니다. 누르면 해당 게시물로 새창이동합니다.">
																	</a>
								<a href="#" onclick="return false;" class="btn_view_zoom"><span class="ico i_zoom_vw"></span>이미지보기</a>
								    							<a href="#" class="btn_view_video" id="imageVideo_11609756" onclick="return false;">
    								<span class="badge_vdo">동영상 재생</span>
    							</a>
    																				</div>
						<div class="prod_info">
							<p class="prod_name">
								<strong class="pop_rank"><span class="screen_out">인기 순위</span>1</strong>									<a href="http://prod.danawa.com/info/?pcode=11609756&amp;cate=112758" target="_blank" onmousedown="_trkEventLog('15상품리스트_상품명')" name="productName">
										한성컴퓨터 올데이롱 TFX5470H									</a>
																<span class="head_ico_wrap" style="">
																	<a href="http://brand.danawa.com/dsnote" class="head_ico hico_notebook" target="_blank">표준노트북</a>&nbsp;</span>
							</p>
							
							<dl class="prod_spec_set">
								<dt class="screen_out">상세 스펙</dt>
								<dd>
									<div class="spec_list">
										<a href="#" class="view_dic" onclick="jQuery.termDicViewLink(1476,'view',this,8); return false;">39.62cm(15.6인치)</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(1707,'view',this,8); return false;">AMD</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(213154,'view',this,8); return false;">라이젠7-3세대</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(213148,'view',this,8); return false;">르누아르</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(213154,'view',this,8); return false;">4800H 2.9GHz(4.2GHz)</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(2604,'view',this,8); return false;">옥타코어</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(1708,'view',this,8); return false;">1920x1080(FHD)</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(210610,'view',this,8); return false;">NTSC</a>: <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(210610,'view',this,8); return false;">72%</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(210571,'view',this,8); return false;">sRGB</a>: <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(210571,'view',this,8); return false;">100%</a> / 300nit / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(3871,'view',this,8); return false;">광시야각(IPS)</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(361,'view',this,8); return false;">눈부심방지</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(29767,'view',this,8); return false;">슈퍼브라이트</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(26549,'view',this,8); return false;">슬림형 베젤</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(23556,'view',this,8); return false;">DDR4</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(288,'view',this,8); return false;">16GB</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(25742,'view',this,8); return false;">M.2(NVMe)</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(2,'view',this,8); return false;">SSD</a>: <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(2,'view',this,8); return false;">500GB</a> / Radeon Graphics / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(213733,'view',this,8); return false;">기가비트 유선랜</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(213739,'view',this,8); return false;">802.11ax(Wi-Fi 6)</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(3515,'view',this,8); return false;">블루투스 5.1</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(290,'view',this,8); return false;">HDMI</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(212773,'view',this,8); return false;">웹캠(HD)</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(25931,'view',this,8); return false;">USB Type-C</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(2794,'view',this,8); return false;">USB 3.0</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(295,'view',this,8); return false;">USB 2.0</a> / MicroSD카드 / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(335,'view',this,8); return false;">키보드 라이트</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(31513,'view',this,8); return false;">USB-PD</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(1990,'view',this,8); return false;">91.24Wh</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(313,'view',this,8); return false;">운영체제</a>: <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(324,'view',this,8); return false;">미포함(프리도스)</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(25877,'view',this,8); return false;">두께</a>: <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(25877,'view',this,8); return false;">16.8mm</a> / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(1478,'view',this,8); return false;">1.5kg</a> / 일반유통상품 / <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(28127,'view',this,8); return false;">용도</a>: <a href="#" class="view_dic" onclick="jQuery.termDicViewLink(25544,'view',this,8); return false;">사무/인강용</a> / 색상 : 블랙																			</div>
								</dd>
							</dl>
								<div class="relation_goods_unit" style="">
								</div>
															<div class="prod_rel_content" style="">
																	<dl class="rel_item rel_article"><dt>관련기사</dt><dd><a href="http://prod.danawa.com/community/ProdBoardView.php?nSeq=4318513&amp;nBlogCateSeq1=62&amp;blogSection=3&amp;prod_c=11609756&amp;cate=112758" target="_blank" onmousedown="_trkEventLog('15상품리스트_관련기사')">게임&amp;작업 둘 다 포기 못하는 당신을 위해 / 한성컴퓨터 올데이롱 TFX5470H 노트북 리뷰 [노리다]</a></dd></dl></div>
							
							<div class="prod_sub_info">
								<div class="prod_sub_meta">
									<dl class="meta_item mt_date">
										<dt>등록월</dt>
										<dd>2020.06</dd>
									</dl>
																			<dl class="meta_item mt_comment"><dt>상품의견</dt><dd>&nbsp;<a href="/info/?pcode=11609756&amp;cate=112758#bookmark_cm_opinion" target="_blank" onmousedown="_trkEventLog('15상품리스트_상품의견')"><strong>641</strong></a>건</dd></dl><dl class="meta_item"><a href="http://brand.danawa.com/hansung" title="해당상품 브랜드로그 보러가기" target="_blank" onmousedown="_trkEventLog('15상품리스트_브랜드로그')">브랜드로그</a></dl><dl class="meta_item">
																					<dt>
																								<a href="#" title="관심상품에 담기" onclick="javascript:$.danawaAddProductToWishList(this, 11609756, 'list'); return false;" onmousedown="_trkEventLog('15상품리스트_담기')">
													관심상품
												</a>
												<input type="hidden" id="wishListBundleVal_11609756" value="//한성컴퓨터 올데이롱 TFX5470H//0">
											</dt>
																				</dl>
									
								</div>
							</div>
						</div>
						<div class="prod_pricelist" data-cate="860|869|31883|0">
															<ul>
																		<li id="productInfoDetail_11609756">
										<p class="chk_sect">
											32몰
																							<label for="prodCompareCheck_11609756"><input type="checkbox" onmousedown="_trkEventLog('15상품리스트_비교')" id="prodCompareCheck_11609756"><strong>상품비교</strong></label>
																						<input type="hidden" name="compareValue" value="11609756">
											<input type="hidden" name="compareRepValue" value="11609756">
										</p>

										<p class="price_sect">
																																		<a href="http://prod.danawa.com/info/?pcode=11609756&amp;cate=112758" target="_blank" onmousedown="_trkEventLog('15상품리스트_가격')">
													<strong>925,000</strong>원
												</a>
																						<span class="more_btn_wrap">
												<button class="ico i_more">가격정보 더보기</button>
												<span class="layer_price_more" id="layer_price_more_11609756"></span>
												<span class="sep_line"></span>
											</span>
																					</p>

											<input type="hidden" id="min_price_11609756" value="925,000">

											<div class="over_preview">
												<p class="memory_sect">
																									<a href="http://prod.danawa.com/info/?pcode=11609756&amp;cate=112758" target="_blank" onmousedown="_trkEventLog('15상품리스트_가격')">
												
																								</a>
												SSD 500GB												</p>
											</div>
									</li>
																	</ul>
																					</div>

					</div>
				</li>
"""

In [ ]:
soup = BeautifulSoup(html)
a = soup.select_one('p.prod_name a')
len(a)